# Train a RoBERTa model

In [1]:
import sys

sys.path.append("..")

In [2]:
from transformers import (
    DataCollatorWithPadding,
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizerFast,
    TrainingArguments,
)

from adna.pylib import consts
from adna.pylib.adna_dataset import ADnaDataset
from adna.pylib.weighted_trainer import WeightedTrainer

## Data augmentation parameters

In [14]:
REV_COMP_RATE = 0.2
TO_N_RATE = 0.02

## Training parameters

In [3]:
TRAIN_EPOCHS = 20
LEARNING_RATE = 3e-5
TRAIN_BATCH_SIZE = 96
EVAL_BATCH_SIZE = 96

MODEL_DIR = 'train'  # Save models to this sub-directory

## Get the tokenizer

In [4]:
tokenizer_path = str(consts.MT_DIR)
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)

## Get the datasets

In [5]:
train_dataset = ADnaDataset(
    'train',
    tokenizer,
    rev_comp_rate=REV_COMP_RATE,
    to_n_rate=TO_N_RATE,
    # limit=TRAIN_BATCH_SIZE,
)
eval_dataset = ADnaDataset('val', tokenizer)  # , limit=EVAL_BATCH_SIZE)

In [6]:
train_dataset[0]

{'input_ids': [0, 2293, 1568, 1125, 285, 2218, 729, 267, 50, 262, 2028, 56, 50, 374, 50, 341, 2850, 3352, 3984, 1302, 640, 277, 3774, 283, 1006, 275, 948, 285, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': tensor(0)}

In [7]:
len(train_dataset)

287233

## Build the model

In [8]:
config = RobertaConfig(
    vocab_size=consts.VOCAB_SIZE,
    num_hidden_layers=6,
    type_vocab_size=1,
    num_labels=len(train_dataset.weights),
    max_position_embeddings=consts.MAX_LENGTH,
)

In [9]:
model = RobertaForSequenceClassification(config=config)

## Build the data collator

In [10]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=consts.MAX_LENGTH,
)

## Build the trainer

In [11]:
training_args = TrainingArguments(
    output_dir=consts.MT_DIR / MODEL_DIR,
    overwrite_output_dir=True,
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    seed=23,
)

In [12]:
trainer = WeightedTrainer(
    train_dataset,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

## Train

In [13]:
trainer.train()

/museum/rafe/other/adna/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287233
  Num Epochs = 20
  Instantaneous batch size per device = 96
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 59860


Epoch,Training Loss,Validation Loss
1,0.376300,0.244983
2,0.290900,0.215134
3,0.261400,0.212256
4,0.243100,0.184775
5,0.228500,0.176542
6,0.217200,0.179326
7,0.208400,0.171200
8,0.198900,0.156981
9,0.190900,0.153605
10,0.183800,0.159917


***** Running Evaluation *****
  Num examples = 95744
  Batch size = 96
Saving model checkpoint to ../data/UF46992/train/checkpoint-2993
Configuration saved in ../data/UF46992/train/checkpoint-2993/config.json
Model weights saved in ../data/UF46992/train/checkpoint-2993/pytorch_model.bin
tokenizer config file saved in ../data/UF46992/train/checkpoint-2993/tokenizer_config.json
Special tokens file saved in ../data/UF46992/train/checkpoint-2993/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 95744
  Batch size = 96
Saving model checkpoint to ../data/UF46992/train/checkpoint-5986
Configuration saved in ../data/UF46992/train/checkpoint-5986/config.json
Model weights saved in ../data/UF46992/train/checkpoint-5986/pytorch_model.bin
tokenizer config file saved in ../data/UF46992/train/checkpoint-5986/tokenizer_config.json
Special tokens file saved in ../data/UF46992/train/checkpoint-5986/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 95744
  Ba

TrainOutput(global_step=59860, training_loss=0.20022459836297715, metrics={'train_runtime': 43479.1579, 'train_samples_per_second': 132.124, 'train_steps_per_second': 1.377, 'total_flos': 1.48628938742808e+17, 'train_loss': 0.20022459836297715, 'epoch': 20.0})

### It looks like epoch ?? is best

Path = `../data/UF46992/models/checkpoint-??`